In [1]:
# Basics
import numpy as np
import pandas as pd
import pickle
import h5py

/Users/anders1991/miniconda3/envs/py3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
name = "cnn_reg9"
classification = False
timesteps = 1
batch_size = 3
# batch_size = 3
nsamples = 50
# nsamples = 3

In [3]:
with open("./data/partition.pkl", 'rb') as f:
    partition = pickle.load(f)
    
partition["train"] = partition["train"]
partition["valid"] = partition["valid"]

In [7]:
len(partition["train"]) + len(partition["valid"]) + len(partition["test"])

1389

# Load labels

In [91]:
target = pd.read_csv("./data/ERU_Scores_Ids_5-Scans_Validity-0_VisuallyScored.csv")
labels = target.set_index("StId").to_dict()["ERU.M2"]

In [87]:
if classification:
    # combine 0+1 as 0 = no emph in scan, 1 = no emph in region
    label_converter = {0: 0, 1: 0, 2: 1, 3: 2, 4: 3, 5: 4, 6: 5}
else:
    # Rescale to [0;1]; merge category 0+1
    # label_converter = {0: 0.0, 1: 0.0, 2: 0.03, 3: 0.155, 4: 0.38, 5: 0.63, 6: 0.88}
    label_converter = {0: 0, 1: 0, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6}
labels = {key: label_converter[val] for key, val in labels.items()}

# Load data

In [51]:
from scripts.data_gen import DataGenerator

Using TensorFlow backend.


In [53]:
gen = DataGenerator(labels, partition, mode="valid", oversample=False, 
                    classes=1, batch_size=batch_size, timesteps=timesteps, cropped=True)

# Load model

In [8]:
from keras.models import load_model
from glob import glob

In [9]:
path = sorted(glob("./output/"+name+"/weights/*.hdf5"))[-1]
print(path)

model = load_model(path)

./output/rnn_reg3/weights/epoch_001-valloss_0.82-valacc_0.19.hdf5


In [10]:
print(model.summary(line_length=120))

________________________________________________________________________________________________________________________
Layer (type)                           Output Shape               Param #       Connected to                            
input_1 (InputLayer)                   (None, 5, 1, 50, 146, 118) 0                                                     
________________________________________________________________________________________________________________________
TD_mpool3D (TimeDistributed)           (None, 5, 1, 50, 146, 118) 0             input_1[0][0]                           
________________________________________________________________________________________________________________________
TD_conv_1 (TimeDistributed)            (None, 5, 6, 50, 142, 114) 156           TD_mpool3D[0][0]                        
________________________________________________________________________________________________________________________
TD_ReLU_2 (TimeDistributed)     

In [14]:
model0 = load_model("./output/cnn_reg6/weights/epoch_038-valloss_0.68-valacc_0.38.hdf5")

# Predict

In [11]:
cd scripts

/Users/anders1991/deepbreath/scripts


In [12]:
# X, y = predGen.__next__()
X, y = gen.__getitem__(0) # select batch index
y_ = model.predict_on_batch(X)

In [13]:
y_

array([[0.8944244 ],
       [0.89624983],
       [0.89250004]], dtype=float32)

In [31]:
y_

array([[0.8965243 ],
       [0.89590216],
       [0.89304775]], dtype=float32)

In [ ]:
# X, y = predGen.__next__()
# X0, y0 = gen.__getitem__(0) # select batch index
y0_ = model0.predict_on_batch(X)

In [ ]:
if classification:
    actual = np.argmax(y, axis=1)
    # actual = np.squeeze(y).astype(int)
    predicted = np.argmax(y_, axis=1)
    
    comp = np.column_stack((actual, predicted))
else:
    comp0 = np.empty((batch_size,2))
    comp0[:,0] = np.reshape(y, (batch_size))
    comp0[:,1] = np.reshape(np.round(y0_), (batch_size))
    
    comp = np.empty((batch_size,2))
    comp[:,0] = np.reshape(y, (batch_size))
    comp[:,1] = np.reshape(np.round(y_), (batch_size))

# left: actual, right: predicted
print(comp0)
print()
print(comp)

[[ 0.  1.]
 [ 0. -0.]
 [ 0.  0.]
 [ 0.  1.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  1.]
 [ 0.  0.]
 [ 0.  1.]
 [ 0.  0.]
 [ 3.  1.]
 [ 0.  1.]
 [ 0.  0.]
 [ 3.  1.]
 [ 0.  0.]
 [ 2.  1.]
 [ 0.  1.]
 [ 2.  0.]
 [ 2.  1.]
 [ 0.  0.]
 [ 2.  1.]
 [ 2.  0.]
 [ 0.  0.]
 [ 2.  1.]
 [ 0.  0.]
 [ 2.  1.]
 [ 0.  0.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0. -0.]
 [ 0.  0.]
 [ 3.  1.]
 [ 0.  0.]
 [ 0.  0.]
 [ 4.  1.]
 [ 0.  1.]
 [ 2.  0.]
 [ 0.  1.]
 [ 2.  0.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  0.]
 [ 0.  0.]
 [ 2.  1.]
 [ 0.  0.]
 [ 0.  1.]
 [ 0.  0.]
 [ 4.  1.]
 [ 2.  1.]]


In [14]:
if classification:
    actual = np.argmax(y, axis=1)
    # actual = np.squeeze(y).astype(int)
    predicted = np.argmax(y_, axis=1)
    
    comp = np.column_stack((actual, predicted))
else:
    comp = np.empty((batch_size,2))
    comp[:,0] = np.reshape(y, (batch_size))
    comp[:,1] = np.reshape(np.round(y_), (batch_size))

# left: actual, right: predicted
print(comp)

[[ 0.  1.]
 [ 0. -0.]
 [ 0.  0.]
 [ 0.  1.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  1.]
 [ 0.  0.]
 [ 0.  1.]
 [ 0.  0.]
 [ 3.  1.]
 [ 0.  1.]
 [ 0.  0.]
 [ 3.  1.]
 [ 0.  0.]
 [ 2.  1.]
 [ 0.  1.]
 [ 2.  0.]
 [ 2.  1.]
 [ 0.  0.]
 [ 2.  1.]
 [ 2.  0.]
 [ 0.  0.]
 [ 2.  1.]
 [ 0.  0.]
 [ 2.  1.]
 [ 0.  0.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0. -0.]
 [ 0.  0.]
 [ 3.  1.]
 [ 0.  0.]
 [ 0.  0.]
 [ 4.  1.]
 [ 0.  1.]
 [ 2.  0.]
 [ 0.  1.]
 [ 2.  0.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  0.]
 [ 0.  0.]
 [ 2.  1.]
 [ 0.  0.]
 [ 0.  1.]
 [ 0.  0.]
 [ 4.  1.]
 [ 2.  1.]]
